In [1]:
key = 13
vocab = [char for char in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ-']


def encrypt(text):
    """Returns the encrypted form of 'text'."""
    indexes = [vocab.index(char) for char in text]
    encrypted_indexes = [(idx + key) % len(vocab) for idx in indexes]
    encrypted_chars = [vocab[idx] for idx in encrypted_indexes]
    encrypted = ''.join(encrypted_chars)
    return encrypted

print(encrypt('THIS-IS-A-SECRET'))

FUVEMVEMNMERPDRF


In [2]:
import random
import torch

message_length = 32

def dataset(num_examples):
    
    """Returns a list of 'num_examples' pairs of the form (encrypted, original).

    Both elements of the pair are tensors containing indexes of each character
    of the corresponding encrypted or original message.
    """
    dataset = []
    for x in range(num_examples):
        ex_out = ''.join([random.choice(vocab) for x in range(message_length)])
        # may be: MANR-TQNNAFEGIDE-OXQZANSVEMJXWSU
        ex_in = encrypt(''.join(ex_out))
        # may be: ZN-DMFC--NSRTVQRMAJCLN-EHRZWJIEG
        ex_in = [vocab.index(x) for x in ex_in]
        # may be: [25, 13, 26, 3, 12, 5, 2, 26, 26, ...
        ex_out = [vocab.index(x) for x in ex_out]
        # may be: [12, 0, 13, 17, 26, 19, 16, 13, ...
        dataset.append([torch.tensor(ex_in), torch.tensor(ex_out)])
    return dataset

### Create embedding

In [3]:
vocab_size = len(vocab)
embedding_dim = 5

# Step 1
embed = torch.nn.Embedding(vocab_size, embedding_dim)

In [4]:
hidden_dim = 10

# Step 1
lstm = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)

In [5]:
def zero_hidden(hidden_dim):
    return (torch.zeros(1, 32, hidden_dim),
            torch.zeros(1, 32, hidden_dim))

In [6]:
# Use an affine transformation to go return the right size
linear = torch.nn.Linear(hidden_dim, vocab_size)

In [7]:
# Create loss
loss_fn = torch.nn.CrossEntropyLoss()

In [8]:
# Optimizer
optimizer = torch.optim.Adam(list(embed.parameters()) + 
                             list(lstm.parameters()) +
                             list(linear.parameters()), lr=0.001)

### Main loop

In [9]:
num_epochs = 10
num_examples = 128

accuracies, max_accuracy = [], 0

for x in range(num_epochs):
    print('Epoch: {}'.format(x))
    for encrypted, original in dataset(num_examples):
        
#         optimizer.zero_grad()
        
        # encrypted.size() = [64]
        lstm_in = embed(encrypted)
        # lstm_in.size() = [64, 5]. This is a 2D tensor, but LSTM expects 
        # a 3D tensor. So we insert a fake dimension.
        lstm_in = lstm_in.unsqueeze(1)
        
        # lstm_in.size() = [64, 1, 5]
        # Get outputs from the LSTM.
        lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden(hidden_dim))
        # lstm_out.size() = [64, 1, 10]
        # Apply the affine transform.
        scores = linear(lstm_out)
        # scores.size() = [64, 1, 27], but loss_fn expects a tensor
        # of size [64, 27, 1]. So we switch the second and third dimensions.
        scores = scores.transpose(1, 2)
        # original.size() = [64], but original should also be a 2D tensor
        # of size [64, 1]. So we insert a fake dimension.
        original = original.unsqueeze(1)
        # Calculate loss.
        loss = loss_fn(scores, original)
        
        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
    print('Loss: {:6.4f}'.format(loss.item()))

Epoch: 0
Loss: 3.0144
Epoch: 1
Loss: 2.2816
Epoch: 2
Loss: 1.4830
Epoch: 3
Loss: 0.8648
Epoch: 4
Loss: 0.5933
Epoch: 5
Loss: 0.3174
Epoch: 6
Loss: 0.2276
Epoch: 7
Loss: 0.1489
Epoch: 8
Loss: 0.0890
Epoch: 9
Loss: 0.0442


In [14]:
with torch.no_grad():
        matches, total = 0, 0
        for encrypted, original in dataset(num_examples):
            lstm_in = embed(encrypted)
            lstm_in = lstm_in.unsqueeze(1)
            lstm_out, lstm_hidden = lstm(lstm_in, zero_hidden(hidden_dim))
            scores = linear(lstm_out)
            # Compute a softmax over the outputs
            predictions = torch.softmax(scores, dim=2)
            # Choose the letter with the maximum probability
            _, batch_out = predictions.max(dim=2)
            # Remove fake dimension
            batch_out = batch_out.squeeze(1)
            # Calculate accuracy
            matches += torch.eq(batch_out, original).sum().item()
            total += torch.numel(batch_out)
        accuracy = matches / total
        print('Accuracy: {:4.2f}%'.format(accuracy * 100))

Accuracy: 100.00%
